In [55]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import utils
import json
import os
import git
import warnings
warnings.filterwarnings("ignore")

In [56]:
from dotenv import load_dotenv
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY

## Get the necessary data from Hopsworks

In [57]:
project = hopsworks.login()
fs = project.get_feature_store() 
# secrets = utils.secrets_api(project.name)

CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

# latitude =
# longitude =

today = datetime.datetime.now()

2025-01-22 09:21:28,733 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-22 09:21:28,748 INFO: Initializing external client
2025-01-22 09:21:28,749 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-22 09:21:30,099 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


In [58]:
# Retrieve feature groups
bike_fg = fs.get_feature_group(
    name='bike_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)


## Clone and pull the repository with the bike data

In [59]:
# Configuration
REPO_URL = "https://github.com/MaxHalford/bike-sharing-history"
CLONE_DIR = "./bike_data/bike-sharing-history"
TARGET_CITY = "dublin"
FILE_NAME = "jcdecaux.geojson"

In [60]:
if not os.path.exists(CLONE_DIR):
    print("Cloning repository...")
    git.Repo.clone_from(REPO_URL, CLONE_DIR)
repo = git.Repo(CLONE_DIR)

# go to main and pull the latest changes
repo.git.checkout("main", force=True)
repo.remotes.origin.pull()

## Get the latest datetime present in the bike data

In [61]:
# last_bike_datetime = "2025-01-06 15:06:11 UTC"

bike_df = bike_fg.read()

last_bike_datetime = bike_df["datetime"].max()
last_bike_datetime = last_bike_datetime.strftime("%Y-%m-%d %H:%M:%S %Z")

last_bike_datetime

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.91s) 


'2025-01-21 00:00:00 UTC'

## Loop through the commits and convert the bike data into a dataframe

In [62]:
results = {}

# Populate the results dict with the stations
data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
if os.path.exists(data_file):
    with open(data_file, "r") as f:
        data = f.read()
        data = json.loads(data)
        for feature in data["features"]:
            name = feature["properties"]["name"]
            results[name] = []

In [63]:
start_time = datetime.datetime.now()
last_day_and_hour = None

for commit in repo.iter_commits():
    # Stop when we reach the earliest bike date
    if commit.committed_datetime <= datetime.datetime.strptime(last_bike_datetime, "%Y-%m-%d %H:%M:%S %Z").replace(tzinfo=datetime.timezone.utc):
        print("breaking at: ", commit.committed_datetime)
        break

    # Skip commits from today
    if commit.committed_datetime > today.replace(tzinfo=datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0):
        first_commited_datetime = repo.commit().committed_datetime
        continue

    day_and_hour = commit.committed_datetime.replace(minute=0, second=0, microsecond=0)
    # print("day_and_hour: ", day_and_hour, " - last_day_and_hour: ", last_day_and_hour)
    if day_and_hour == last_day_and_hour:
        continue
    last_day_and_hour = day_and_hour

    # Get the data for the commit
    print("Processing commit: ", commit.hexsha, " - ", commit.committed_datetime)
    try:
        repo.git.checkout(commit.hexsha, force=True)
    except Exception as e:
        print("Error checking out commit: ", e)
        break

    data_file = os.path.join(CLONE_DIR, "data/stations/" + TARGET_CITY + "/" + FILE_NAME)
    if os.path.exists(data_file):
        with open(data_file, "r") as f:
            data = f.read()
            data = json.loads(data)
            for feature in data["features"]:
                try:
                    results[feature["properties"]["name"]].append([feature["properties"]["available_bikes"], commit.committed_datetime])
                except KeyError:
                    continue

print(results)


Processing commit:  7d7e74f082b6f20060a18140ed3f3757196e3c07  -  2025-01-21 23:49:01+00:00
Processing commit:  77397012b399a7312597b931a7918a6e7a4a0c1c  -  2025-01-21 22:49:11+00:00
Processing commit:  affc9bccb78d07e0a927ff15d24f89a69f44ac6c  -  2025-01-21 21:49:02+00:00
Processing commit:  11a181a3f06d45e271debb40b401235a0f77ae66  -  2025-01-21 20:49:14+00:00
Processing commit:  e1523ff6677bce7af71743101b92cec87ed9842b  -  2025-01-21 19:49:03+00:00
Processing commit:  139f9648ee0cff9504c18d9e91a2d57b72ee8b40  -  2025-01-21 18:49:26+00:00
Processing commit:  01722bad66c21731e72a6e21e5a03ad60e64d459  -  2025-01-21 17:49:13+00:00
Processing commit:  27661d072cfeefabd4d28591383ef6f43cd35678  -  2025-01-21 16:50:09+00:00
Processing commit:  61a7ae153ef7f4ed7ef82f33806e20e3b86ffd21  -  2025-01-21 15:49:09+00:00
Processing commit:  ac06c0a24305133574123f2605172013877ce88b  -  2025-01-21 14:49:09+00:00
Processing commit:  80aa46ebd0a5ae49f14f00a79909556c1de05dd9  -  2025-01-21 13:49:09+00:00

In [64]:
# turn results into a dataframe
df_bike_today = pd.DataFrame()

for station, values in results.items():
    if len(values) > 0:
        df = pd.DataFrame(values, columns=["num_bikes_available", "datetime"])
        df["datetime"] = pd.to_datetime(df["datetime"], utc=True)
        df["station"] = station.replace(" ", "_")
        df_bike_today = pd.concat([df_bike_today, df])

# if empty, do nothing
if not df_bike_today.empty:
    df_bike_today.dropna(inplace=True)
    df_bike_today["num_bikes_available"] = df_bike_today["num_bikes_available"].astype("float32")
    df_bike_today = df_bike_today[df_bike_today['station'].isin([STATION.replace(" ", "_")])]
    df_bike_today['datetime'] = pd.to_datetime(df_bike_today['datetime'], utc=True).dt.floor('H') + datetime.timedelta(hours=1)
df_bike_today


,num_bikes_available,datetime,station
0,40.0,2025-01-22 00:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
1,40.0,2025-01-21 23:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
2,39.0,2025-01-21 22:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
3,40.0,2025-01-21 21:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
4,39.0,2025-01-21 20:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
5,40.0,2025-01-21 19:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
6,40.0,2025-01-21 18:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
7,17.0,2025-01-21 17:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
8,2.0,2025-01-21 16:00:00+00:00,HEUSTON_BRIDGE_(NORTH)
9,2.0,2025-01-21 15:00:00+00:00,HEUSTON_BRIDGE_(NORTH)


## Fetch the weather data for the same time period

In [65]:
forecast_df = utils.get_hourly_weather_forecast(CITY)
forecast_df = forecast_df.rename(columns={'date_x': 'datetime'})
forecast_df = forecast_df.drop(columns=['date_y', 'date_only'])
forecast_df.dropna(inplace=True)

print(forecast_df.empty)

forecast_df

features: {'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
params: {'latitude': 53.35, 'longitude': -6.26, 'hourly': ['temperature_2m', 'apparent_temperature', 'rain', 'snowfall', 'wind_speed_10m'], 'daily': ['daylight_duration', 'rain_sum']}
Coordinates 53.5°N -6.25°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
False


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2025-01-22 00:00:00+00:00,5.30,2.372532,0.0,0.0,10.464797,30252.697266,0.900000,dublin
1,2025-01-22 01:00:00+00:00,5.15,2.287240,0.1,0.0,10.028439,30252.697266,0.900000,dublin
2,2025-01-22 02:00:00+00:00,4.95,2.126904,0.1,0.0,9.693295,30252.697266,0.900000,dublin
3,2025-01-22 03:00:00+00:00,4.65,1.882041,0.1,0.0,9.114471,30252.697266,0.900000,dublin
4,2025-01-22 04:00:00+00:00,4.10,1.322361,0.1,0.0,8.647496,30252.697266,0.900000,dublin
...,...,...,...,...,...,...,...,...,...
235,2025-01-31 19:00:00+00:00,7.60,4.567654,0.3,0.0,15.034041,32065.236328,5.700001,dublin
236,2025-01-31 20:00:00+00:00,7.15,3.668416,0.3,0.0,16.923830,32065.236328,5.700001,dublin
237,2025-01-31 21:00:00+00:00,6.70,2.504890,0.3,0.0,20.674156,32065.236328,5.700001,dublin
238,2025-01-31 22:00:00+00:00,6.25,1.161357,0.3,0.0,25.478743,32065.236328,5.700001,dublin


## Insert the bike and weather data into Hopsworks

In [66]:
if df_bike_today.empty:
    print("No bike data available for today")
else:
    bike_fg.insert(df_bike_today)

Uploading Dataframe: 100.00% |██████████| Rows 24/24 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


In [67]:
if forecast_df.empty:
    print("No weather forecast available for today")
else:
    weather_fg.insert(forecast_df, write_options={"wait_for_job": True})


Uploading Dataframe: 100.00% |██████████| Rows 240/240 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions
2025-01-22 09:22:51,513 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-22 09:22:54,694 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-22 09:24:55,839 INFO: Waiting for log aggregation to finish.
2025-01-22 09:28:52,052 INFO: Execution finished successfully.
